## Get download links for h5ad files from HuBMAP and SenNet (HRApop)

## Install and import libraries

In [1]:

%pip install pandas

import pandas as pd
import os

Note: you may need to restart the kernel to use updated packages.


## Global settings

In [2]:
hra_pop_version = 'v0.11.1'

## Load data and extract Entity IDs

In [3]:
sankey_full = pd.read_csv(
    f'https://raw.githubusercontent.com/x-atlas-consortia/hra-pop/refs/heads/main/output-data/{hra_pop_version}/reports/universe-ad-hoc/sankey.csv')

sankey_full

C:\Users\abueckle\AppData\Local\Temp\1\ipykernel_33780\960230895.py:1: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  sankey_full = pd.read_csv(


,portal,study_paper,doi,lead_author,is_azimuth_reference,donor_id,donor_sex,donor_age,donor_development_stage,donor_race,...,unique_dataset_id,link_to_h5ad_file,sc_transcriptomics_or_sc_proteomics,cell_type_annotation_tool,omap_id,number_of_cells_total,number_of_unique_cell_types,hubmap_dataset_publication_status,is_rui_registered,is_atlas_dataset
0,KPMP,NaN,http://dx.doi.org/10.1681/ASN.2016091027,NaN,NaN,Donor1,Male,NaN,NaN,NaN,...,http://dx.doi.org/10.1681/ASN.2016091027#Donor...,http://dx.doi.org/10.1681/ASN.2016091027,NaN,NaN,NaN,NaN,NaN,NaN,True,False
1,KPMP,NaN,http://dx.doi.org/10.1681/ASN.2016091027,NaN,NaN,Donor2,Male,NaN,NaN,NaN,...,http://dx.doi.org/10.1681/ASN.2016091027#Donor...,http://dx.doi.org/10.1681/ASN.2016091027,NaN,NaN,NaN,NaN,NaN,NaN,True,False
2,KPMP,NaN,http://dx.doi.org/10.1681/ASN.2016091027,NaN,NaN,Donor3,Male,NaN,NaN,NaN,...,http://dx.doi.org/10.1681/ASN.2016091027#Donor...,http://dx.doi.org/10.1016/j.trsl.2017.07.006,NaN,NaN,NaN,NaN,NaN,NaN,True,False
3,HRA,NaN,NaN,NaN,NaN,Donor1,Female,38.0,NaN,NaN,...,http://purl.org/ccf/1.5/omap-1#Donor1_TissueBl...,https://hubmapconsortium.github.io/ccf-release...,NaN,NaN,NaN,NaN,NaN,NaN,True,False
4,"HIRN, ESPACE",NaN,https://doi.org/10.5281/zenodo.7742474,NaN,NaN,Donor1,Male,NaN,NaN,NaN,...,http://purl.org/ccf/1.5/omap-6#Donor1_TissueBl...,https://zenodo.org/record/7742474,NaN,NaN,NaN,NaN,NaN,NaN,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16811,SPARC,NaN,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,Donor1,Female,49.0,NaN,NaN,...,https://sparc.science/datasets/390/version/1?d...,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,NaN,NaN,NaN,NaN,True,False
16812,SPARC,NaN,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,Donor1,Female,49.0,NaN,NaN,...,https://sparc.science/datasets/390/version/1?d...,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,NaN,NaN,NaN,NaN,True,False
16813,SPARC,NaN,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,Donor1,Female,49.0,NaN,NaN,...,https://sparc.science/datasets/390/version/1?d...,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,NaN,NaN,NaN,NaN,True,False
16814,SPARC,NaN,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,Donor1,Female,49.0,NaN,NaN,...,https://sparc.science/datasets/390/version/1?d...,https://sparc.science/datasets/390/version/1?d...,NaN,NaN,NaN,NaN,NaN,NaN,True,False


In [4]:
# filter, only keep sc-transcriptomics data that was run through at least one cell type annotation tool
sankey = sankey_full[((sankey_full['cell_type_annotation_tool']).notna()) & (sankey_full['cell_type_annotation_tool'] != 'sc_proteomics')]
sankey

,portal,study_paper,doi,lead_author,is_azimuth_reference,donor_id,donor_sex,donor_age,donor_development_stage,donor_race,...,unique_dataset_id,link_to_h5ad_file,sc_transcriptomics_or_sc_proteomics,cell_type_annotation_tool,omap_id,number_of_cells_total,number_of_unique_cell_types,hubmap_dataset_publication_status,is_rui_registered,is_atlas_dataset
134,CxG,Emphysema Cell Atlas,https://doi.org/10.1016/j.immuni.2023.01.032,Chaoqun Wang,NaN,Emp1,Female,NaN,NaN,unknown,...,https://api.cellxgene.cziscience.com/dp/v1/col...,https://cellxgene.cziscience.com/collections/0...,NaN,azimuth,NaN,33891.0,69.0,NaN,False,False
135,CxG,Emphysema Cell Atlas,https://doi.org/10.1016/j.immuni.2023.01.032,Chaoqun Wang,NaN,Emp1,Female,NaN,NaN,unknown,...,https://api.cellxgene.cziscience.com/dp/v1/col...,https://cellxgene.cziscience.com/collections/0...,NaN,celltypist,NaN,33891.0,69.0,NaN,False,False
136,CxG,Emphysema Cell Atlas,https://doi.org/10.1016/j.immuni.2023.01.032,Chaoqun Wang,NaN,Emp1,Female,NaN,NaN,unknown,...,https://api.cellxgene.cziscience.com/dp/v1/col...,https://cellxgene.cziscience.com/collections/0...,NaN,popv,NaN,33891.0,69.0,NaN,False,False
137,CxG,Emphysema Cell Atlas,https://doi.org/10.1016/j.immuni.2023.01.032,Chaoqun Wang,NaN,Emp2,Female,NaN,NaN,unknown,...,https://api.cellxgene.cziscience.com/dp/v1/col...,https://cellxgene.cziscience.com/collections/0...,NaN,azimuth,NaN,27804.0,70.0,NaN,False,False
138,CxG,Emphysema Cell Atlas,https://doi.org/10.1016/j.immuni.2023.01.032,Chaoqun Wang,NaN,Emp2,Female,NaN,NaN,unknown,...,https://api.cellxgene.cziscience.com/dp/v1/col...,https://cellxgene.cziscience.com/collections/0...,NaN,celltypist,NaN,27804.0,70.0,NaN,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16656,SenNet,NaN,NaN,NaN,NaN,NaN,Male,42.0,NaN,White,...,https://entity.api.sennetconsortium.org/entiti...,https://data.sennetconsortium.org/dataset?uuid...,NaN,popv,NaN,11310.0,30.0,NaN,False,False
16676,SenNet,NaN,NaN,NaN,NaN,NaN,Female,54.0,NaN,White,...,https://entity.api.sennetconsortium.org/entiti...,https://data.sennetconsortium.org/dataset?uuid...,NaN,azimuth,NaN,21596.0,47.0,NaN,False,False
16677,SenNet,NaN,NaN,NaN,NaN,NaN,Female,54.0,NaN,White,...,https://entity.api.sennetconsortium.org/entiti...,https://data.sennetconsortium.org/dataset?uuid...,NaN,popv,NaN,21596.0,47.0,NaN,False,False
16742,SenNet,NaN,NaN,NaN,NaN,NaN,Female,39.0,NaN,White,...,https://entity.api.sennetconsortium.org/entiti...,https://data.sennetconsortium.org/dataset?uuid...,NaN,azimuth,NaN,8436.0,29.0,NaN,False,False


In [5]:
# extract UUIDs
# https://entity.api.hubmapconsortium.org/entities/ebaa609a1819b22767471082d7baa0d9

def get_uuids(column:pd.core.series.Series, effort:str):
  """Gets UUIDs from dataset IDs

  Args:
      column (pd.core.series.Series): a Pandas series
      effort (str): effort name

  Returns:
      result: list with UUIDs
  """
  result = set(dataset_id.split('/')[len(dataset_id.split('/'))-1]
            for dataset_id in column if effort in dataset_id)
  return result


hubmap_ids = get_uuids(sankey['dataset_id'], 'hubmap')
sennet_ids = get_uuids(sankey['dataset_id'], 'sennet')
print(f'hubmap_ids: {len(hubmap_ids)}, sennet_ids: {len(sennet_ids)}')

hubmap_ids: 395, sennet_ids: 17


## Build URLs for download, export

In [6]:
assets_api_hubmap = 'https://assets.hubmapconsortium.org/'
assets_api_sennet = 'https://assets.api.sennetconsortium.org/'
filename = 'expr.h5ad'

result = {
  'effort':[],
  'uuid': [],
  'download_url':[]
}

def assemble_url(ids:list, url:str, effort:str):
  for id in ids:
    result['effort'].append(effort)
    result['uuid'].append(id)
    result['download_url'].append(f'{url}{id}/{filename}')  
    
assemble_url(hubmap_ids, assets_api_hubmap, 'hubmap')
assemble_url(sennet_ids, assets_api_sennet, 'sennet')

df_result = pd.DataFrame(result)
df_result

,effort,uuid,download_url
0,hubmap,853ab5348f619043ab2f997ac8ae14f0,https://assets.hubmapconsortium.org/853ab5348f...
1,hubmap,56cbda4789f04d79c0c3dffe21816d48,https://assets.hubmapconsortium.org/56cbda4789...
2,hubmap,87e85403e3f7a946fc40bdcf5cba8360,https://assets.hubmapconsortium.org/87e85403e3...
3,hubmap,79f8c7ec1b200dfe5b46d0bcfec02bd7,https://assets.hubmapconsortium.org/79f8c7ec1b...
4,hubmap,e901621d0400c18d3d69decd46a82c84,https://assets.hubmapconsortium.org/e901621d04...
...,...,...,...
407,sennet,2f26357b4ba8bbe545c77ee9f91cafda,https://assets.api.sennetconsortium.org/2f2635...
408,sennet,ae5d324e38aedcda9ab2aba5f1267e6f,https://assets.api.sennetconsortium.org/ae5d32...
409,sennet,fd80ead9309975aa3ffd4d8456a7706f,https://assets.api.sennetconsortium.org/fd80ea...
410,sennet,2eeab84ac0e040e4c8bd2ddd0709fd6c,https://assets.api.sennetconsortium.org/2eeab8...


In [7]:
# export to csv
df_result.to_csv('download_urls.csv', index=False)

## Download h5ad files

In [8]:
# Make sure the data folder is present
folder_path = "data"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Folder '{folder_path}' created.")
else:
    print(f"Folder '{folder_path}' already exists.")

def download_h5ad(uuid:str, url:str):
  """Downloads h5ad files given download URL and UUID

  Args:
      uuid (str): UUID
      url (str): download URL
  """
  # Define the path to the file. 
  file_path = f'{folder_path}/{uuid}.h5ad'

    # Check if the file exists
  if not os.path.exists(file_path):
      # If the file doesn't exist, run the curl command
      !curl -L {url} -o {file_path}
      print(f"File downloaded and saved at {file_path}")
  else:
      print(f"File already exists at {file_path}")

Folder 'data' created.


In [9]:
# loop through df and download if needed
for uuid, download_url in zip(df_result['uuid'], df_result['download_url']):
  print(f'Attempting to download: {download_url}')
  download_h5ad(uuid, download_url)

Attempting to download: https://assets.hubmapconsortium.org/853ab5348f619043ab2f997ac8ae14f0/expr.h5ad
File downloaded and saved at data/853ab5348f619043ab2f997ac8ae14f0.h5ad
Attempting to download: https://assets.hubmapconsortium.org/56cbda4789f04d79c0c3dffe21816d48/expr.h5ad


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
 16  216M   16 36.4M    0     0  48.5M      0  0:00:04 --:--:--  0:00:04 48.8M
 44  216M   44 96.1M    0     0  54.8M      0  0:00:03  0:00:01  0:00:02 54.9M
 58  216M   58  125M    0     0  45.7M      0  0:00:04  0:00:02  0:00:02 45.7M
 71  216M   71  155M    0     0  41.4M      0  0:00:05  0:00:03  0:00:02 41.5M
 86  216M   86  186M    0     0  39.2M      0  0:00:05  0:00:04  0:00:01 39.2M
 98  216M   98  212M    0     0  36.9M      0  0:00:05  0:00:05 --:--:-- 35.2M
100  216M  100  216M    0     0  36.7M      0  0:00:05  0:00:05 --:--:-- 29.0M


^C
File downloaded and saved at data/56cbda4789f04d79c0c3dffe21816d48.h5ad
Attempting to download: https://assets.hubmapconsortium.org/87e85403e3f7a946fc40bdcf5cba8360/expr.h5ad
File downloaded and saved at data/87e85403e3f7a946fc40bdcf5cba8360.h5ad
Attempting to download: https://assets.hubmapconsortium.org/79f8c7ec1b200dfe5b46d0bcfec02bd7/expr.h5ad


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  4  202M    4 8623k    0     0  7304k      0  0:00:28  0:00:01  0:00:27 7326k
 10  202M   10 20.5M    0     0  9752k      0  0:00:21  0:00:02  0:00:19 9767k
 15  202M   15 32.0M    0     0  10.1M      0  0:00:19  0:00:03  0:00:16 10.1M
 23  202M   23 46.5M    0     0  11.1M      0  0:00:18  0:00:04  0:00:14 11.1M
 30  202M   30 61.1M    0     0  11.8M      0  0:00:17  0:00:05  0:00:12 13.3M
 39  202M   39 79.6M    0     0  12.9M      0  0:00:15  0:00:06  0:00:09 14.2M
 49  202M   49  100M    0     0  14.0M      0  0:00:14  0:00:07  0:00:07 16.0M
 62  202M   62  125M    0     0  15.3M      0  0:00:13  0:00:08  0:00:05 18.7M
 76  202M   76  154M    0     0  16.8M      0  0:00

: 

: 